In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.applications import VGG16, ResNet50
from efficientnet.tfkeras import EfficientNetB5
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

import numpy as np
import random
import cv2
import matplotlib.pyplot as plt

from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method

os.environ["TF_METAL_ENABLED"] = "1"

In [2]:
from run_models import MyDataset, MyModel, vgg16_training, effnetB5_training, resnet50_training, ensemble_training


In [3]:
mydata = MyDataset('preprocessed', (224, 224), processed_data_dir='dataset_mnist_train_2000_test_100')

Load from:  dataset_mnist_train_2000_test_100/train_images.npy
Load from:  dataset_mnist_train_2000_test_100/test_images.npy
Load from:  dataset_mnist_train_2000_test_100/train_labels.npy
Load from:  dataset_mnist_train_2000_test_100/test_labels.npy
Load from:  dataset_mnist_train_2000_test_100/train_images_fgsm_vgg16.npy
Load from:  dataset_mnist_train_2000_test_100/train_images_fgsm_resnet50.npy
Load from:  dataset_mnist_train_2000_test_100/test_images_fgsm_vgg16.npy
Load from:  dataset_mnist_train_2000_test_100/test_images_fgsm_resnet50.npy


### VGG16


In [4]:
vgg16 = vgg16_training(mydata, model_name_set_to='vgg16',  preload_model='saved_models_train_2000_test_100/mnist_vgg16_model.h5')
vgg16_fgsm_trained = vgg16_training(mydata, model_name_set_to='vgg16_fgsm_trained', preload_model='saved_models_train_2000_test_100/mnist_vgg16_model_with_fgsm_vgg16.h5')

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-05-22 01:55:16.383170: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-22 01:55:16.383200: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 21140042 (80.64 MB)
Trainable params: 6425354 (24.51 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________
saved_models_train_2000_test_100/mnist_vgg16_model.h5 Model Loaded!
Model: "sequential"
_____________________________________________________

Test vgg16 model with original test images.

In [5]:
train_loss, train_accuracy = vgg16.model.evaluate(x=mydata.train_images, y=mydata.train_labels, batch_size=128, verbose=0)
print(f'mnist_vgg16_model - Train loss: {train_loss}. Train Accuracy: {train_accuracy}')

test_loss, test_accuracy = vgg16.model.evaluate(x=mydata.test_images, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'mnist_vgg16_model - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

2023-05-22 01:55:24.343276: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


mnist_vgg16_model - Train loss: 0.010438578203320503. Train Accuracy: 0.9994999766349792
mnist_vgg16_model - Test loss: 0.026431698352098465. Test Accuracy: 1.0


Test vgg16 model with adversarial test images.

In [6]:
test_loss, test_accuracy = vgg16.model.evaluate(x=mydata.test_images_fgsm_vgg16, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'mnist_vgg16_model on self-generated FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

mnist_vgg16_model on self-generated FGSM attrack data - Test loss: 1.9391045570373535. Test Accuracy: 0.4099999964237213


Test vgg16_fgsm_trained model with original and adversarial test images.

In [7]:
train_loss, train_accuracy = vgg16_fgsm_trained.model.evaluate(x=mydata.train_images_fgsm_vgg16, y=mydata.train_labels, batch_size=128, verbose=0)
print(f'vgg16_fgsm_trained - Train loss: {train_loss}. Train Accuracy: {train_accuracy}')

test_loss, test_accuracy = vgg16_fgsm_trained.model.evaluate(x=mydata.test_images, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_fgsm_trained - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')
test_loss, test_accuracy = vgg16_fgsm_trained.model.evaluate(x=mydata.test_images_fgsm_vgg16, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_fgsm_trained on self-generated FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

2023-05-22 01:55:54.466878: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


vgg16_fgsm_trained - Train loss: 0.2762700617313385. Train Accuracy: 0.9135000109672546
vgg16_fgsm_trained - Test loss: 0.0426224023103714. Test Accuracy: 1.0
vgg16_fgsm_trained on self-generated FGSM attrack data - Test loss: 0.35638272762298584. Test Accuracy: 0.8799999952316284


In [8]:
test_loss, test_accuracy = vgg16_fgsm_trained.model.evaluate(x=mydata.test_images_fgsm_resnet50, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_fgsm_trained on resnet50 FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

vgg16_fgsm_trained on resnet50 FGSM attrack data - Test loss: 0.7982957363128662. Test Accuracy: 0.7200000286102295


### ResNet50

In [9]:
resnet50 = resnet50_training(mydata, model_name_set_to='resnet50',  preload_model='saved_models_train_2000_test_100/mnist_resnet50_model.h5')
resnet50_fgsm_trained = resnet50_training(mydata, model_name_set_to='resnet50_fgsm_trained', preload_model='saved_models_train_2000_test_100/mnist_resnet50_model_with_fgsm_resnet50.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 49280650 (187.99 MB)
Trainable params: 25692938 (98.01 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________
saved_models_train_2000_test_100/mnist_resnet50_model.h5 Model Loaded!
Model: "sequential"
________________________________________________

Test resnet50 model with original test images.

In [10]:
train_loss, train_accuracy = resnet50.model.evaluate(x=mydata.train_images, y=mydata.train_labels, batch_size=128, verbose=0)
print(f'mnist_resnet50_model - Train loss: {train_loss}. Train Accuracy: {train_accuracy}')

test_loss, test_accuracy = resnet50.model.evaluate(x=mydata.test_images, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'mnist_resnet50_model - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

2023-05-22 01:56:29.100369: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


mnist_resnet50_model - Train loss: 0.22230477631092072. Train Accuracy: 0.9465000033378601
mnist_resnet50_model - Test loss: 0.2559686601161957. Test Accuracy: 0.9300000071525574


Test resnet50 model with adversarial test images.

In [11]:
test_loss, test_accuracy = resnet50.model.evaluate(x=mydata.test_images_fgsm_resnet50, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'mnist_resnet50_model on self-generated FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

mnist_resnet50_model on self-generated FGSM attrack data - Test loss: 4.515739917755127. Test Accuracy: 0.11999999731779099


Test resnet50_fgsm_trained model with original and adversarial test images.

In [12]:
train_loss, train_accuracy = resnet50_fgsm_trained.model.evaluate(x=mydata.train_images_fgsm_resnet50, y=mydata.train_labels, batch_size=128, verbose=0)
print(f'resnet50_fgsm_trained - Train loss: {train_loss}. Train Accuracy: {train_accuracy}')

test_loss, test_accuracy = resnet50_fgsm_trained.model.evaluate(x=mydata.test_images, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'resnet50_fgsm_trained - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

test_loss, test_accuracy = resnet50_fgsm_trained.model.evaluate(x=mydata.test_images_fgsm_resnet50, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'resnet50_fgsm_trained on self-generated FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

2023-05-22 01:56:45.356500: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


resnet50_fgsm_trained - Train loss: 0.5554242134094238. Train Accuracy: 0.8144999742507935
resnet50_fgsm_trained - Test loss: 0.8940862417221069. Test Accuracy: 0.6299999952316284
resnet50_fgsm_trained on self-generated FGSM attrack data - Test loss: 0.6128746867179871. Test Accuracy: 0.7699999809265137


### VGG16 Ensemble


In [13]:
vgg16_ensemble_trained = vgg16_training(mydata, model_name_set_to='vgg16_ensemble_trained',  preload_model='saved_model_vgg16_ensemble_vgg16_fgsm_trained_resnet50_fgsm_trained/mnist_vgg16_model_ensemble_adversarial.h5')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 21140042 (80.64 MB)
Trainable params: 6425354 (24.51 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________
saved_model_vgg16_ensemble_vgg16_fgsm_trained_resnet50_fgsm_trained/mnist_vgg16_model_ensemble_adversarial.h5 Model Loaded!


In [14]:
test_loss, test_accuracy = vgg16_ensemble_trained.model.evaluate(x=mydata.test_images, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_ensemble_trained - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

2023-05-22 01:56:59.524030: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


vgg16_ensemble_trained - Test loss: 0.06635254621505737. Test Accuracy: 0.9599999785423279


In [15]:
test_loss, test_accuracy = vgg16_ensemble_trained.model.evaluate(x=mydata.test_images_fgsm_vgg16, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_ensemble_trained on VGG16 FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

test_loss, test_accuracy = vgg16_ensemble_trained.model.evaluate(x=mydata.test_images_fgsm_resnet50, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_ensemble_trained on resnet50 FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

vgg16_ensemble_trained on VGG16 FGSM attrack data - Test loss: 0.04391443356871605. Test Accuracy: 0.9900000095367432
vgg16_ensemble_trained on resnet50 FGSM attrack data - Test loss: 0.8983032703399658. Test Accuracy: 0.7300000190734863


In [25]:
test_adv_images_generated_resnet50_fgsm_trained = np.load('dataset_mnist_train_2000_test_100_resnet50_fgsm_trained/test_images_fgsm_resnet50.npy')
test_adv_images_generated_vgg16_fgsm_trained = np.load('dataset_mnist_train_2000_test_100_vgg16_fgsm_trained/test_images_fgsm_vgg16.npy')
test_adv_images_generated_vgg16_ensemble_trained = np.load('dataset_mnist_train_2000_test_100_vgg16_ensemble_trained/test_images_fgsm_vgg16.npy')

test_adv_images_generated_resnet101_fgsm = np.load('dataset_mnist_train_2000_test_100/test_images_fgsm_resnet101.npy')
test_adv_images_generated_resnet101_pgd = np.load('dataset_mnist_train_2000_test_100/test_images_pgd_resnet101.npy')

In [29]:
test_loss, test_accuracy = resnet50_fgsm_trained.model.evaluate(x=test_adv_images_generated_resnet50_fgsm_trained, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'resnet50_fgsm_trained on resnet50_fgsm_trained FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

test_loss, test_accuracy = resnet50_fgsm_trained.model.evaluate(x=test_adv_images_generated_vgg16_fgsm_trained, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'resnet50_fgsm_trained on vgg16_fgsm_trained FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

test_loss, test_accuracy = resnet50_fgsm_trained.model.evaluate(x=test_adv_images_generated_vgg16_ensemble_trained, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'resnet50_fgsm_trained on vgg16_ensemble_trained FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

test_loss, test_accuracy = resnet50_fgsm_trained.model.evaluate(x=test_adv_images_generated_resnet101_fgsm, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'resnet50_fgsm_trained on resnet101 FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

test_loss, test_accuracy = resnet50_fgsm_trained.model.evaluate(x=test_adv_images_generated_resnet101_pgd, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'resnet50_fgsm_trained on resnet101 PGD attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

resnet50_fgsm_trained on resnet50_fgsm_trained FGSM attrack data - Test loss: 3.3058838844299316. Test Accuracy: 0.1599999964237213
resnet50_fgsm_trained on vgg16_fgsm_trained FGSM attrack data - Test loss: 0.5773460268974304. Test Accuracy: 0.800000011920929
resnet50_fgsm_trained on vgg16_ensemble_trained FGSM attrack data - Test loss: 0.5644710659980774. Test Accuracy: 0.7900000214576721
resnet50_fgsm_trained on resnet101 FGSM attrack data - Test loss: 0.7307191491127014. Test Accuracy: 0.7599999904632568
resnet50_fgsm_trained on resnet101 PGD attrack data - Test loss: 0.8732733130455017. Test Accuracy: 0.6399999856948853


In [30]:
test_loss, test_accuracy = vgg16_fgsm_trained.model.evaluate(x=test_adv_images_generated_resnet50_fgsm_trained, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_fgsm_trained on resnet50_fgsm_trained FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

test_loss, test_accuracy = vgg16_fgsm_trained.model.evaluate(x=test_adv_images_generated_vgg16_fgsm_trained, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_fgsm_trained on vgg16_fgsm_trained FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

test_loss, test_accuracy = vgg16_fgsm_trained.model.evaluate(x=test_adv_images_generated_vgg16_ensemble_trained, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_fgsm_trained on vgg16_ensemble_trained FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

test_loss, test_accuracy = vgg16_fgsm_trained.model.evaluate(x=test_adv_images_generated_resnet101_fgsm, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_fgsm_trained on resnet101 FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

test_loss, test_accuracy = vgg16_fgsm_trained.model.evaluate(x=test_adv_images_generated_resnet101_pgd, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_fgsm_trained on resnet101 PGD attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

vgg16_fgsm_trained on resnet50_fgsm_trained FGSM attrack data - Test loss: 0.8694702386856079. Test Accuracy: 0.6899999976158142
vgg16_fgsm_trained on vgg16_fgsm_trained FGSM attrack data - Test loss: 0.7436202764511108. Test Accuracy: 0.7799999713897705
vgg16_fgsm_trained on vgg16_ensemble_trained FGSM attrack data - Test loss: 0.6193116903305054. Test Accuracy: 0.800000011920929
vgg16_fgsm_trained on resnet101 FGSM attrack data - Test loss: 1.0418117046356201. Test Accuracy: 0.6299999952316284
vgg16_fgsm_trained on resnet101 PGD attrack data - Test loss: 0.06937707215547562. Test Accuracy: 1.0


In [31]:
test_loss, test_accuracy = vgg16_ensemble_trained.model.evaluate(x=test_adv_images_generated_resnet50_fgsm_trained, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_ensemble_trained on resnet50_fgsm_trained FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

test_loss, test_accuracy = vgg16_ensemble_trained.model.evaluate(x=test_adv_images_generated_vgg16_fgsm_trained, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_ensemble_trained on vgg16_fgsm_trained FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

test_loss, test_accuracy = vgg16_ensemble_trained.model.evaluate(x=test_adv_images_generated_vgg16_ensemble_trained, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_ensemble_trained on vgg16_ensemble_trained FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

test_loss, test_accuracy = vgg16_ensemble_trained.model.evaluate(x=test_adv_images_generated_resnet101_fgsm, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_ensemble_trained on resnet101 FGSM attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

test_loss, test_accuracy = vgg16_ensemble_trained.model.evaluate(x=test_adv_images_generated_resnet101_pgd, y=mydata.test_labels, batch_size=128, verbose=0)
print(f'vgg16_ensemble_trained on resnet101 PGD attrack data - Test loss: {test_loss}. Test Accuracy: {test_accuracy}')

vgg16_ensemble_trained on resnet50_fgsm_trained FGSM attrack data - Test loss: 0.9242722988128662. Test Accuracy: 0.6800000071525574
vgg16_ensemble_trained on vgg16_fgsm_trained FGSM attrack data - Test loss: 0.07603666931390762. Test Accuracy: 0.9599999785423279
vgg16_ensemble_trained on vgg16_ensemble_trained FGSM attrack data - Test loss: 0.07602505385875702. Test Accuracy: 0.9900000095367432
vgg16_ensemble_trained on resnet101 FGSM attrack data - Test loss: 0.8631994724273682. Test Accuracy: 0.7200000286102295
vgg16_ensemble_trained on resnet101 PGD attrack data - Test loss: 0.1221960037946701. Test Accuracy: 0.949999988079071
